In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import json
import os
import sys

# -----------------------------
# 1️⃣ Load Model and Labels - FIXED PATHS
# -----------------------------
model_path = r"/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/notebooks/face_recognition_attendance_final.keras"
labels_path = r"/home/sandeshprasai/Final_Semester_Project/AI_Attendance_System/ai-ml-model/notebooks/class_labels.json"

print("🔍 Checking file paths...")
print(f"Model path: {model_path}")
print(f"Model exists: {os.path.exists(model_path)}")
print(f"Labels path: {labels_path}")
print(f"Labels exist: {os.path.exists(labels_path)}")

# Check if files exist before trying to load
if not os.path.exists(model_path):
    print(f"❌ Cannot find model at {model_path}")
    print(f"Files in directory: {os.listdir(os.path.dirname(model_path)) if os.path.exists(os.path.dirname(model_path)) else 'Directory not found'}")
    sys.exit()

if not os.path.exists(labels_path):
    print(f"❌ Cannot find class labels at {labels_path}")
    sys.exit()

try:
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    sys.exit()

try:
    with open(labels_path, "r") as f:
        labels = json.load(f)
    print(f"✅ Class labels loaded. Number of classes: {len(labels)}")
except Exception as e:
    print(f"❌ Error loading labels: {e}")
    sys.exit()

index_to_name = {v: k for k, v in labels.items()}

# -----------------------------
# 2️⃣ Preprocess Input Image
# -----------------------------
def preprocess_image(image_path, target_size=(224,224)):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Cannot read image {image_path}")
        return None, None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, target_size)
    img_array = np.expand_dims(img_resized, axis=0)
    img_array = img_array / 255.0
    return img_array, img_resized

# -----------------------------
# 3️⃣ Generate Grad-CAM
# -----------------------------
def generate_gradcam(model, img_array, class_index, last_conv_layer_name=None):
    if last_conv_layer_name is None:
        for layer in reversed(model.layers):
            if "conv" in layer.name:
                last_conv_layer_name = layer.name
                break
    
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]
    
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap + 1e-8)
    
    return heatmap.numpy()

# -----------------------------
# 4️⃣ Overlay Heatmap
# -----------------------------
def overlay_heatmap(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, colormap)
    overlayed = cv2.addWeighted(img, 1-alpha, heatmap_color, alpha, 0)
    return overlayed

# -----------------------------
# 5️⃣ Process All Images in Folder - FIXED PATH
# -----------------------------
folder_path = "/home/sandeshprasai/Newfolder"  # Fixed path
supported_extensions = (".jpg", ".jpeg", ".png", ".bmp")

print(f"\n📁 Checking image folder: {folder_path}")
print(f"Folder exists: {os.path.exists(folder_path)}")

if not os.path.exists(folder_path):
    print(f"❌ Image folder not found: {folder_path}")
    print(f"Available folders in home: {os.listdir('/home/sandeshprasai')}")
    sys.exit()

image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(supported_extensions)]
print(f"📸 Found {len(image_files)} images to process")

if not image_files:
    print("❌ No images found in the folder")
    sys.exit()

print("\n📋 Predictions:\n" + "-"*50)
print(f"{'Filename':<25} | {'Predicted':<20} | Confidence")
print("-"*50)

for filename in image_files:
    image_path = os.path.join(folder_path, filename)
    print(f"🔄 Processing: {filename}")
    
    img_array, img_resized = preprocess_image(image_path)
    if img_array is None:
        continue
    
    preds = model.predict(img_array, verbose=0)  # verbose=0 to reduce output
    class_idx = int(np.argmax(preds[0]))
    predicted_name = index_to_name.get(class_idx, "Unknown")
    confidence = preds[0][class_idx] * 100
    
    # Print prediction
    print(f"{filename:<25} | {predicted_name:<20} | {confidence:6.2f}%")
    
    # Generate Grad-CAM
    try:
        heatmap = generate_gradcam(model, img_array, class_idx)
        overlayed_img = overlay_heatmap(img_resized, heatmap)
        
        # Display
        plt.figure(figsize=(8, 6))
        plt.imshow(overlayed_img)
        plt.axis('off')
        plt.title(f"{filename}\nPredicted: {predicted_name} ({confidence:.2f}%)", fontsize=12)
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"⚠️  Could not generate Grad-CAM for {filename}: {e}")
        # Show original image instead
        plt.figure(figsize=(6, 6))
        plt.imshow(img_resized)
        plt.axis('off')
        plt.title(f"{filename}\nPredicted: {predicted_name} ({confidence:.2f}%)", fontsize=12)
        plt.tight_layout()
        plt.show()

print("\n✅ Processing complete!")